In [9]:
# # Packages 

import os 

import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
%matplotlib inline

#data, labels and classes 
classes= ['0','1']
data= 'data/spectrograms/'
labels='data/labels_extension.txt'


In [8]:
# # Data loader 

class SpectrogramDataset(Dataset):
    def __init__(self, path_dataset, path_labels):
        self.labels= open(path_labels, 'r').read().split('\n')
        self.path_dataset = path_dataset
        self.images = os.listdir(path_dataset)
    
    def __getitem__(self, index):
        image_return = np.array(Image(self.path_dataset + self.images[index]))
        label= labels[index]
        return torch.from_numpy(image), label
    
    def __len__(self):
        return len(self.labels)



dataset_s = SpectrogramDataset(data, labels)